In [25]:
!pip install scikit-surprise

In [35]:
data = pd.read_csv('amazon.tsv', sep='\t', nrows=50_000)
data.sample(1)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_date
11981,US,32621511,R3SOZW66QSY7XV,0385333854,710547068,Woody Guthrie: A Life,Books,5,11,12,N,N,The greatest biography ever written,2005-10-07


In [75]:
from surprise.model_selection import RandomizedSearchCV

# Creamos un objeto Reader
reader = Reader(rating_scale=(1, 5))

# Cargamos el dataset
dataset = Dataset.load_from_df(data[['customer_id', 'product_id', 'star_rating']], reader)

param_dist = {'n_factors': [30, 40, 50, 100, 150],
              'lr_all': [0.002, 0.005, 0.01, 0.1, 0.02],
              'reg_all': [0.02, 0.04, 0.06,0.01, 0.9]}
random_search = RandomizedSearchCV(SVD, param_dist, measures=['rmse'], cv=3, n_iter=5)
random_search.fit(dataset)
# Mejores resultados
print("Mejores resultados:")
print(random_search.best_score['rmse'])
print(random_search.best_params['rmse'])

# Detalles de cada ejecución
print("\nDetalles de cada ejecución:")
results_df = pd.DataFrame.from_dict(random_search.cv_results)
print(results_df)

Mejores resultados:
1.1717085678102002
{'n_factors': 100, 'lr_all': 0.02, 'reg_all': 0.02}

Detalles de cada ejecución:
   split0_test_rmse  split1_test_rmse  split2_test_rmse  mean_test_rmse  \
0          1.177426          1.188593          1.167902        1.177974   
1          1.171056          1.183190          1.160879        1.171709   
2          1.178740          1.189830          1.168922        1.179164   
3          1.173072          1.186563          1.163892        1.174509   
4          1.195536          1.206248          1.185370        1.195718   

   std_test_rmse  rank_test_rmse  mean_fit_time  std_fit_time  mean_test_time  \
0       0.008456               3       0.748349      0.121960        0.144452   
1       0.009120               1       0.866601      0.024259        0.118849   
2       0.008541               4       0.862636      0.008654        0.107534   
3       0.009311               2       0.861004      0.013323        0.207780   
4       0.008524        

In [79]:
from surprise.model_selection import RandomizedSearchCV
from surprise.prediction_algorithms.knns import KNNWithMeans

param_dist = {'k': [20, 30, 40],
              'sim_options': {'name': ['cosine', 'pearson'],
                              'user_based': [True, False]}}

random_search = RandomizedSearchCV(KNNWithMeans, param_dist, measures=['rmse'], cv=3, n_iter=5)
random_search.fit(dataset)

# Mejores resultados
print("Mejores resultados:")
print(random_search.best_score['rmse'])
print(random_search.best_params['rmse'])

# Detalles de cada ejecución
print("\nDetalles de cada ejecución:")
results_df = pd.DataFrame.from_dict(random_search.cv_results)
print(results_df)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computin

In [76]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Dividimos en conjunto de entrenamiento y conjunto de prueba
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

# Creamos el modelo SVD (Singular Value Decomposition)
#model = SVD()
model = SVD(n_factors=40, lr_all=0.02, reg_all=0.01)

# Entrenamos el modelo
model.fit(trainset)

# Evaluamos el modelo
predictions = model.test(testset)

# Calculamos RMSE (Root Mean Squared Error)
accuracy.rmse(predictions)



RMSE: 1.1454


1.1453618094656748

In [77]:
# Definimos una función para obtener las recomendaciones para un customer_id
def get_recommendations(customer_id, n=10):
    # Creamos una lista de todos los productos
    all_products = data['product_id'].unique().tolist()
    # Removemos los productos que el usuario ya ha visto
    products_seen = data[data['customer_id'] == customer_id]['product_id'].tolist()
    products_to_predict = [product for product in all_products if product not in products_seen]

    # Predecimos ratings para los productos que el usuario no ha visto
    predictions = [model.predict(customer_id, product) for product in products_to_predict]

    # Ordenamos las predicciones por rating (de mayor a menor)
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Devolvemos las primeras n predicciones
    top_n = sorted_predictions[:n]
    recommended_products = [pred.iid for pred in top_n]

    return recommended_products

# Ahora, para obtener las recomendaciones para un customer_id específico
customer_id = 'tu_customer_id'
recommendations = get_recommendations(customer_id)

print("Recomendaciones para el customer_id", customer_id, ":")
for i, product_id in enumerate(recommendations, 1):
    print(f"{i}. {product_id}")


Recomendaciones para el customer_id tu_customer_id :
1. 0595346677
2. 0974786918
3. 0345476581
4. 0897934474
5. 0312343337
6. 1419607553
7. 1412045320
8. 0060283262
9. 0316734934
10. 0740754939


In [78]:
# Definimos una función para obtener las recomendaciones para un customer_id
def get_recommendations(customer_id, n=10):
    # Creamos una lista de todos los productos
    all_products = data[['product_id', 'product_title']].drop_duplicates()

    # Removemos los productos que el usuario ya ha visto
    products_seen = data[data['customer_id'] == customer_id]['product_id'].tolist()
    all_products = all_products[~all_products['product_id'].isin(products_seen)]

    # Predecimos ratings para los productos que el usuario no ha visto
    predictions = [model.predict(customer_id, product_id) for product_id in all_products['product_id']]

    # Ordenamos las predicciones por rating (de mayor a menor)
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Devolvemos las primeras n predicciones con sus títulos
    top_n = sorted_predictions[:n]
    recommended_products = [(pred.iid, all_products[all_products['product_id'] == pred.iid]['product_title'].values[0]) for pred in top_n]

    return recommended_products

# Ahora, para obtener las recomendaciones para un customer_id específico
customer_id = 'tu_customer_id'
recommendations = get_recommendations(customer_id)

print("Recomendaciones para el customer_id", customer_id, ":")
for i, (product_id, product_title) in enumerate(recommendations, 1):
    print(f"{i}. {product_title} (ID: {product_id})")


Recomendaciones para el customer_id tu_customer_id :
1. Steven (ID: 0595346677)
2. Jeanne Carley's Ferret Calendar 2006, Ferret Babies (ID: 0974786918)
3. The Underdog: How I Survived the World's Most Outlandish Competitions (ID: 0345476581)
4. How to Spot a Dangerous Man Before You Get Involved (ID: 0897934474)
5. Tiny Dancer: The Incredible True Story of a Young Burn Victim's Journey from Afghanistan (ID: 0312343337)
6. She Did That (ID: 1419607553)
7. Self-Change Hypnosis (ID: 1412045320)
8. If You Give a Pig a Party (ID: 0060283262)
9. The Lincoln Lawyer: A Novel (Mickey Haller) (ID: 0316734934)
10. Famous Pairs: A Deliciously Absurd Collection of Portraits (ID: 0740754939)
